In [2]:
#!pip install python-telegram-bot

In [1]:
import telegram
from binance.client import Client

In [4]:
API_KEY = os.getenv('BINANCE_API_KEY')
API_SECRET = os.getenv('BINANCE_API_SECRET')
TELEGRAM_API_TOKEN = os.getenv('TELEGRAM_API_TOKEN')

bot = telegram.Bot(TELEGRAM_API_TOKEN)
client = Client(API_KEY,API_SECRET)

In [3]:
print(bot.get_me())

{'id': 1287326681, 'first_name': 'Test Bot', 'is_bot': True, 'username': 'Test151196Bot', 'can_join_groups': True, 'can_read_all_group_messages': False, 'supports_inline_queries': True}


In [4]:
from telegram.ext import Updater
updater = Updater(token=TELEGRAM_API_TOKEN, use_context=True)

In [5]:
import logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                     level=logging.INFO)

In [6]:
def start(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text="I'm a bot, please talk to me!")

In [7]:
from telegram.ext import CommandHandler
dispatcher = updater.dispatcher
start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

In [13]:
from datetime import datetime as dt
dt.now()

datetime.datetime(2020, 7, 31, 9, 33, 11, 37842)

In [17]:
client.futures_create_order(symbol="ETHUSDT", side="SELL",type="LIMIT",timeInForce="GTC",quantity=0.01,price=326,timestamp=dt.now())

{'orderId': 2467501182,
 'symbol': 'ETHUSDT',
 'status': 'NEW',
 'clientOrderId': 'eZcgAAxCyObICjhyasKUaZ',
 'price': '326',
 'avgPrice': '0.00000',
 'origQty': '0.010',
 'executedQty': '0',
 'cumQty': '0',
 'cumQuote': '0',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'reduceOnly': False,
 'closePosition': False,
 'side': 'SELL',
 'positionSide': 'BOTH',
 'stopPrice': '0',
 'workingType': 'CONTRACT_PRICE',
 'priceProtect': 'false',
 'origType': 'LIMIT',
 'updateTime': 1596168337608}

In [21]:
client.

BinanceAPIException: APIError(code=-2013): Order does not exist.

In [9]:
def echo(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text=update.message.text)

from telegram.ext import MessageHandler, Filters
echo_handler = MessageHandler(Filters.text & (~Filters.command), echo)
dispatcher.add_handler(echo_handler)

In [10]:
j = updater.job_queue

In [24]:
LIMITS = dict({'TRXUSDT':[0.01890,0.01920]})
TEST = False

2020-07-30 22:49:42,530 - JobQueue - ERROR - An uncaught error was raised while executing job callback_minute
Traceback (most recent call last):
  File "c:\users\jeeva\documents\py_projects\general\lib\site-packages\telegram\vendor\ptb_urllib3\urllib3\connectionpool.py", line 617, in urlopen
    chunked=chunked)
  File "c:\users\jeeva\documents\py_projects\general\lib\site-packages\telegram\vendor\ptb_urllib3\urllib3\connectionpool.py", line 402, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "c:\users\jeeva\documents\py_projects\general\lib\site-packages\telegram\vendor\ptb_urllib3\urllib3\connectionpool.py", line 398, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\jeeva\AppData\Local\Programs\Python\Python36\lib\http\client.py", line 1331, in getresponse
    response.begin()
  File "C:\Users\jeeva\AppData\Local\Programs\Python\Python36\lib\http\client.py", line 297, in begin
    version, status, reason = self.

In [16]:
def callback_minute(context: telegram.ext.CallbackContext):
    global LIMITS, TEST
    pos = list(filter(lambda f:(f['symbol'] in list(LIMITS.keys())),client.futures_account()['positions']))
    if pos:
        for position in pos:
            text = ""
            symbol = position['symbol']
            mark_price = client.futures_mark_price(symbol=symbol)['markPrice']
            text += (str(symbol) + " Price : " + str(mark_price))
            text += ".\n"
            text += ("Unrealized Profit : "+ position['unrealizedProfit'])
            text += ".\n"
            text += ("Entry Price : "+ position['entryPrice'])
            if (float(mark_price) > LIMITS[symbol][1]) or (float(mark_price) < LIMITS[symbol][0]):
                context.bot.send_message(chat_id='@jeevantest', text=text)
    if TEST:
        context.bot.send_message(chat_id='@jeevantest', text="Test message")

job_minute = j.run_repeating(callback_minute, interval=60, first=0)

In [18]:
TEST = True

def callback_minute(context: telegram.ext.CallbackContext):
    pos = list(filter(lambda f:(f['symbol']=='XRPUSDT'),client.futures_account()['positions']))
    if pos:
        text = ""
        position = pos[0]
        mark_price = client.futures_mark_price(symbol=position['symbol'])['markPrice']
        text += (str(position['symbol']) + " Price : " + str(mark_price))
        text += ".\n"
        text += ("Unrealized Profit : "+ position['unrealizedProfit'])
        text += ".\n"
        text += ("Entry Price : "+ position['entryPrice'])
        if (float(mark_price) > 0.21) or (float(mark_price) < 0.2050):
            context.bot.send_message(chat_id='@jeevantest', text=text)

job_minute = j.run_repeating(callback_minute, interval=120, first=0)

In [14]:
updater.start_polling()

2020-07-30 19:10:37,036 - JobQueue - ERROR - An uncaught error was raised while executing job callback_minute
Traceback (most recent call last):
  File "c:\users\jeeva\documents\py_projects\general\lib\site-packages\telegram\ext\jobqueue.py", line 418, in tick
    job.run(self._dispatcher)
  File "c:\users\jeeva\documents\py_projects\general\lib\site-packages\telegram\ext\jobqueue.py", line 574, in run
    self.callback(CallbackContext.from_job(self, dispatcher))
  File "<ipython-input-12-97f0d12c8100>", line 3, in callback_minute
    pos = list(filter(lambda f:(f['symbol'] in list(LIMITS.keys())),client.futures_account()['positions']))
  File "c:\users\jeeva\documents\py_projects\general\lib\site-packages\binance\client.py", line 3648, in futures_account
    return self._request_futures_api('get', 'account', True, data=params)
  File "c:\users\jeeva\documents\py_projects\general\lib\site-packages\binance\client.py", line 222, in _request_futures_api
    return self._request(method, ur

In [26]:
from telegram import InlineQueryResultArticle, InputTextMessageContent, InputMediaPhoto
def inline_caps(update, context):
    query = update.inline_query.query
    if not query:
        return
    results = list()
    results.append(
        InlineQueryResultArticle(
            id=query.upper(),
            title='Caps3',
            input_message_content= InputMediaPhoto("https://images.indianexpress.com/2020/07/TEEJ-1-1.jpg")
        )
    )
    context.bot.answer_inline_query(update.inline_query.id, results)

from telegram.ext import InlineQueryHandler
inline_caps_handler3 = InlineQueryHandler(inline_caps)


In [30]:
dispatcher.remove_handler(inline_caps_handler)

In [17]:
j.jobs()

(<telegram.ext.jobqueue.Job at 0x2787e04ec88>,)

In [25]:
for job in j.jobs():
    job.schedule_removal()

2020-07-31 00:52:59,345 - telegram.ext.updater - ERROR - Error while getting Updates: urllib3 HTTPError ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
2020-07-31 00:52:59,358 - telegram.ext.dispatcher - ERROR - No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "c:\users\jeeva\documents\py_projects\general\lib\site-packages\telegram\vendor\ptb_urllib3\urllib3\connectionpool.py", line 617, in urlopen
    chunked=chunked)
  File "c:\users\jeeva\documents\py_projects\general\lib\site-packages\telegram\vendor\ptb_urllib3\urllib3\connectionpool.py", line 402, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "c:\users\jeeva\documents\py_projects\general\lib\site-packages\telegram\vendor\ptb_urllib3\urllib3\connectionpool.py", line 398, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\jeeva\

2020-07-31 00:53:00,401 - telegram.ext.dispatcher - ERROR - No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "c:\users\jeeva\documents\py_projects\general\lib\site-packages\telegram\vendor\ptb_urllib3\urllib3\connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "c:\users\jeeva\documents\py_projects\general\lib\site-packages\telegram\vendor\ptb_urllib3\urllib3\util\connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "C:\Users\jeeva\AppData\Local\Programs\Python\Python36\lib\socket.py", line 745, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\jeeva\documents\py_projects\general\lib\site-packages\telegram\vendo

In [45]:
j.jobs()

(<telegram.ext.jobqueue.Job at 0x1a94f8be0b8>,)